In [1]:
from llm.llm.chatgpt import ChatGPT
from llm.llm.gemini import Gemini
from setup_db import create_chroma_db, connect_to_db, execute_query, DBHUB
from llm_test import reasoning_text2SQL, CoT_reasoning
from llm_general import  find_suitable_row_v2,  get_stock_code_and_suitable_row

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
sql_llm = Gemini()
llm = Gemini('gemini-1.5-flash-8b')

In [3]:
question = "For Vietcombank and Techcombank, please provide the quarterly data for the Debt-to-Equity ratio from 2020 to 2023"

In [4]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
port = '5433'
host = 'localhost'

In [5]:
conn = {
    'user': user,
    'password': password,
    'host': host,
    'port': port,
    'db_name': db_name
}


In [6]:
collection_chromadb = 'category_bank_chroma'
persist_directory = 'data/category_bank_chroma'
bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_non_bank_chroma'
persist_directory = 'data/category_non_bank_chroma'
none_bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_sec_chroma'
persist_directory = 'data/category_sec_chroma'
sec_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_ratio_chroma'
persist_directory = 'data/category_ratio_chroma'
ratio_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'company_name_chroma'
persist_directory = 'data/company_name_chroma'
vector_db_company = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'sql_query'
persist_directory = 'data/sql_query'
vector_db_sql = create_chroma_db(collection_chromadb, persist_directory)

2024-11-09 14:11:04,492 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-09 14:11:04,634 - INFO - Collection category_bank_chroma is not created.
2024-11-09 14:11:05,133 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-09 14:11:05,141 - INFO - Collection category_non_bank_chroma is not created.
2024-11-09 14:11:05,639 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-09 14:11:05,646 - INFO - Collection category_sec_chroma is not created.
2024-11-09 14:11:06,154 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-09 14:11:06,160 - INFO - Collection category_ratio_chroma is not created.
2024-11-09 14:11:06,625 - INFO - Anonymized telemetry enabled. See                     h

In [7]:
db = DBHUB(conn, bank_vector_store, none_bank_vector_store, sec_vector_store, ratio_vector_store, vector_db_company, vector_db_sql)

In [8]:
prompt = "Compare the Return on Assets (ROA) and Return on Equity (ROE) of Vinamilk and Masan Group for the fiscal year 2023 same period."

In [9]:
from llm_general import *
import time

In [10]:
from branch_reasoning import simplify_branch_reasoning

simplify_branch_reasoning(llm, prompt, verbose=True)

2024-11-09 14:11:08,969 - INFO - Completion time of gemini-1.5-flash-8b: 1.8210599422454834s


prompt_token_count: 597
candidates_token_count: 109
total_token_count: 706

Branch reasoning response: 
```json
{
  "steps": [
    "Step 1: Retrieve the stock codes for Vinamilk and Masan Group from the `company_info` table.",
    "Step 2: Extract the ROA and ROE values for Vinamilk and Masan Group from the `financial_ratio` table for the fiscal year 2023, ensuring the correct period selection (quarter 0 for the full year). Compare the extracted ROA and ROE values."
  ]
}
```



['Step 1: Retrieve the stock codes for Vinamilk and Masan Group from the `company_info` table.',
 'Step 2: Extract the ROA and ROE values for Vinamilk and Masan Group from the `financial_ratio` table for the fiscal year 2023, ensuring the correct period selection (quarter 0 for the full year). Compare the extracted ROA and ROE values.']

In [11]:
get_stock_code_and_suitable_row(llm, prompt, db, verbose=True)

2024-11-09 14:11:09,706 - INFO - Completion time of gemini-1.5-flash-8b: 0.7271766662597656s


prompt_token_count: 223
candidates_token_count: 67
total_token_count: 290

Get stock code based on company name response: 
```json
{
    "industry": [],
    "company_name": ["Vinamilk", "Masan Group"],
    "financial_statement_row": [],
    "financial_ratio_row": ["Return on Assets (ROA)", "Return on Equity (ROE)"]
}
```



2024-11-09 14:11:10,908 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-09 14:11:11,390 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Connecting to database test_db, postgres...


2024-11-09 14:11:12,510 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-09 14:11:12,896 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Connecting to database test_db, postgres...


(  stock_code                    company_name  \
 0        VNM    Công ty Cổ phần Sữa Việt Nam   
 1        MSN  Công ty Cổ phần Tập đoàn Masan   
 
                               en_company_name            industry  is_bank  \
 0  Vietnam Dairy Products Joint Stock Company  Food and Beverages    False   
 1                     Masan Group Corporation  Food and Beverages    False   
 
    is_securities  
 0          False  
 1          False  ,
 '\n\nTable `map_category_code_non_bank`\n\nNone\n\nTable `map_category_code_bank`\n\nNone\n\nTable `map_category_code_securities`\n\nNone\n\nTable `map_category_code_ratio`\n\n| ratio_code   | ratio_name                           |\n|:-------------|:-------------------------------------|\n| ER           | Equity Ratio                         |\n| ROI          | Return On Investment                 |\n| DTER         | Debt To Equity Ratio                 |\n| ROA          | Return On Assets                     |\n| ROLTOA       | Return On Long 

In [12]:
llm.reset_token()
llm.usage()

{'input_token': 0, 'output_token': 0}

In [23]:
start = time.time()
history, error_messages, execution_tables = reasoning_text2SQL(llm, prompt, db, 5, sql_llm=sql_llm, verbose=False, branch_reasoning=False)
end = time.time()
print("Time taken: ", end - start)

2024-11-09 14:18:17,515 - INFO - Completion time of gemini-1.5-flash-8b: 0.7344608306884766s


prompt_token_count: 190
candidates_token_count: 67
total_token_count: 257



2024-11-09 14:18:18,077 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-09 14:18:18,477 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Connecting to database test_db, postgres...
Connecting to database test_db, postgres...


2024-11-09 14:18:19,832 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-09 14:18:20,317 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Connecting to database test_db, postgres...


2024-11-09 14:18:21,314 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-09 14:18:23,696 - INFO - Completion time of gemini-1.5-flash-002: 2.377108335494995s


prompt_token_count: 2853
candidates_token_count: 239
total_token_count: 3092

Connecting to database test_db, postgres...
Time taken:  6.942214488983154


In [20]:
for message in history:
    print("==============")
    print(message['role'])
    print('_____________')
    print(message['content'])

system
_____________

    You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query.
    
user
_____________
You have the following database schema:
You're given database about the financial reports of top Vietnamese companies, both bank and cooperates.

All the data in the financial report are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the categories are followed by the International Financial Reporting Standards (IFRS).

There are 3 type of financial reports, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).



Here are the descriptions of tables in the database:



### PostgreSQL tables, with their properties

```sql 

-- Table: company_info

CREATE TABLE company_info(

    stock_code VARCHAR(255) primary key, --The trading symbol.

    is_bank BOOLEAN, --Bool checking wh

In [15]:
last_message = history[-1]['content']
print(last_message)

To construct the SQL query based on the provided natural language query, we will follow the outlined steps.

### Step 1: Retrieve the stock codes for Vinamilk and Masan Group from the `company_info` table.

We know from the provided data that:
- Vinamilk has the stock code `VNM`
- Masan Group has the stock code `MSN`

### Step 2: Extract the ROA and ROE values for Vinamilk and Masan Group from the `financial_ratio` table, filtering for the fiscal year 2023 and the corresponding stock codes identified in Step 1.

We will assume that the ratio codes for ROA and ROE are defined in the `map_category_code_ratio` table, but since we do not have the exact ratio codes from the provided data, we will use placeholders `ROA_code` and `ROE_code` to represent them.

Here’s the SQL query that accomplishes this:

```sql
-- SQL Query to compare ROA and ROE for Vinamilk and Masan Group for the fiscal year 2023
WITH financial_data AS (
    SELECT
        stock_code,
        ratio_code,
        data
    

In [16]:
for message in history:
    print("==============")
    print(message['role'])
    print('_____________')
    print(message['content'])

system
_____________

    You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query.
    
user
_____________
You have the following database schema:
You're given database about the financial reports of top Vietnamese companies, both bank and cooperates.

All the data in the financial report are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the categories are followed by the International Financial Reporting Standards (IFRS).

There are 3 type of financial reports, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).



Here are the descriptions of tables in the database:



### PostgreSQL tables, with their properties

```sql 

-- Table: company_info

CREATE TABLE company_info(

    stock_code VARCHAR(255) primary key, --The trading symbol.

    is_bank BOOLEAN, --Bool checking wh

In [17]:
llm.usage()

{'input_token': 916, 'output_token': 165}

In [18]:
execution_tables[-1]

,stock_code,roa,roe
